# Analysis of NGC 3627 with Optical + X-ray wavelength data

In [60]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import glob
import shutil

from astroquery.mast import Observations

from astropy.io import fits                     # for file handing 
from astropy.table import QTable                # for reading tables
from astropy.coordinates import SkyCoord, FK5   # for defining sky coordinates
import astropy.units as u                       # for using astronomical units (degs)

from XRBID.AutoPhots import RunPhots, DaoFindObjects
from XRBID.WriteScript import WriteReg
from XRBID.Sources import (LoadSources, GetCoords, GetIDs,
                           DaoClean, Crossref)
from XRBID.DataFrameMod import (RemoveElse, FindUnique, BuildFrame,
                                Find)
from XRBID.Align import CorrectAstrometry, CalcPU

from photometry_utils import instrument_info

from regions import CircleSkyRegion, Regions    # for defining sky regions
from regions import TextSkyRegion               # for defining text regions

# define directories to handle data in
cd = os.chdir
hstdir = "/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/HST/"
chandra_hst_dir = "/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/Chandra-HST/"
chandra_jwst_dir = "/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/Chandra-JWST/"
jwstdir = "/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/JWST/"

# jwst directories
f200w = jwstdir+"hlsp_phangs-jwst_jwst_nircam_ngc3627_f200w_v1p1_img.fits"
f300m = jwstdir+"hlsp_phangs-jwst_jwst_nircam_ngc3627_f300m_v1p1_img.fits"
f335m = jwstdir+"hlsp_phangs-jwst_jwst_nircam_ngc3627_f335m_v1p1_img.fits"
f360m = jwstdir+"hlsp_phangs-jwst_jwst_nircam_ngc3627_f360m_v1p1_img.fits"

# hst directories
f275w = hstdir+"M66_mosaic_uvis_f275w_drc_sci.fits"
f336w = hstdir+"M66_mosaic_uvis_f336w_drc_sci.fits"
f438w = hstdir+"M66_mosaic_uvis_f438w_drc_sci.fits"
f555w = hstdir+"M66_mosaic_uvis_f555w_drc_sci.fits"
f814w = hstdir+"M66_mosaic_uvis_f814w_drc_sci.fits"

# Mosaicking
## Querying data with astroquery

In [7]:
filters = ["F275W", "F336W", "F438W", "F555W", "F814W"]
obs_id = [13364, 13477, 15654, 12968]

print("Finding observations for WFC3/UVIS images...")
obs_table = Observations.query_criteria(objectname="M66", obs_collection="HST",
                                        filters=filters,
                                        proposal_id=obs_id, instrument_name="WFC3/UVIS")
print("Done")

Finding observations for WFC3/UVIS images...
Done


In [ ]:
obs_table

In [9]:
%%time
cd(hstdir)
print("Downloading data.\nThis may take several minutes... \n")
products = Observations.get_product_list(obs_table)
data_prod = ['FLC']       
data_type = ['HAP-SVM']   

Observations.download_products(products, project=data_type, 
                               productSubGroupDescription=data_prod)
print("Done")


In [11]:
# Gathering the FLC file names
input_flcs = glob.glob(os.path.join('mastDownload','HST','*','*.fits'))

# Moving files from mastDownload directory into current directory
for flc in input_flcs:
  shutil.copy(flc, os.path.basename(flc))

shutil.rmtree('mastDownload') # removes the extra directories
print("Done")

Done


In [13]:
flc_files = glob.glob("hst*_acs_wfc_*_j*_flc.fits")
flc_files.sort()       # Alphabetizing files

for i,f in enumerate(flc_files):
    try: hdu = fits.open(f)
    except: print(f,"not good. Please remove.")

## F555W
The following code will be run in a python terminal

In [ ]:
from drizzlepac.astrodrizzle import AstroDrizzle as adriz

flc_files = glob.glob("*f555w*flc*.fits")
flc_files.sort()

adriz(flc_files, output='M66_mosaic_uvis_f555w',
      preserve=True, clean=True, context=False,
      skymethod='globalmin+match', # renormalizes the brightness of each field
      driz_cr=True, driz_cr_corr=False,
      final_wcs=True, final_rot=0,  # places North pointed up
      configobj=None, overwrite=False, build=False)  

## F814W

In [ ]:
flc_files = glob.glob("*f814w*flc*.fits")
flc_files.sorqqt()

adriz(flc_files, output='M66_mosaic_uvis_f814w',
      preserve=True, clean=True, context=False,
      skymethod='globalmin+match', # renormalizes the brightness of each field
      driz_cr=True, driz_cr_corr=False,
      final_wcs=True, final_rot=0,  # places North pointed up
      configobj=None, overwrite=False, build=False) 

## F438W

In [ ]:
flc_files = glob.glob("*f438w*flc*.fits")
flc_files.sort()

adriz(flc_files, output='M66_mosaic_uvis_f438w',
      preserve=True, clean=True, context=False,
      skymethod='globalmin+match', # renormalizes the brightness of each field
      driz_cr=True, driz_cr_corr=False,
      final_wcs=True, final_rot=0,  # places North pointed up
      configobj=None, overwrite=False, build=False) 

## F336W

In [ ]:
flc_files = glob.glob("*f336w*flc*.fits")
flc_files.sort()

adriz(flc_files, output='M66_mosaic_uvis_f336w',
      preserve=True, clean=True, context=False,
      skymethod='globalmin+match', # renormalizes the brightness of each field
      driz_cr=True, driz_cr_corr=False,
      final_wcs=True, final_rot=0,  # places North pointed up
      configobj=None, overwrite=False, build=False) 

## F275W

In [ ]:
flc_files = glob.glob("*f275w*flc*.fits")
flc_files.sort()

adriz(flc_files, output='M66_mosaic_uvis_f275w',
      preserve=True, clean=True, context=False,
      skymethod='globalmin+match', # renormalizes the brightness of each field
      driz_cr=True, driz_cr_corr=False,
      final_wcs=True, final_rot=0,  # places North pointed up
      configobj=None, overwrite=False, build=False) 

# Photometry

## F814W

In [ ]:
f814w_data = fits.getdata(f814w)
f814w_data_sliced = f814w_data[3400:3700, 4400:4700]
pixtoarcs = 0.03962
f814w_objects = DaoFindObjects(f814w_data_sliced, fwhm=0.10*2, pixtoarcs=pixtoarcs, threshold=2, plot=True)

# investigating the daofind sources and extracting their photometry
f814w_hdu = fits.open(f814w)
apcorr, aperr, apcorr_ext, aperr_ext = RunPhots(f814w_hdu, gal="M66", instrument="WFC3",
                                               filter='F814W', fwhm_arcs=0.10*2, threshold=2,
                                               num_stars=20, extended_rad=8, reg_correction=[1,1])
f814w_hdu.close()

# ((np.float64(-0.8588325849729528),),
#  np.float64(0.1807607521654574),
#  np.float64(-0.3719457463759386),
#  np.float64(0.10642947110641915))

## F555W

In [ ]:
f555w_data = fits.getdata(f555w)
f555w_data_sliced = f555w_data[3400:3700, 4400:4700]
pixtoarcs = 0.03962
fwhm = 0.18 * 2 # originally found 0.18 arcsec through ds9
f555w_objects = DaoFindObjects(f555w_data_sliced, fwhm=fwhm, pixtoarcs=pixtoarcs,
                               threshold=2, plot=True)

#investigating the daofind sources and extracting their photometry
f555w_hdu = fits.open(f555w)
apcorr555w, aperr555w, apcorr_ext555w, aperr_ext555w = RunPhots(f555w_hdu, gal="M66", instrument="WFC3",
                                               filter='F555W', fwhm_arcs=fwhm, threshold=2,
                                               num_stars=20, extended_rad=8, reg_correction=[1,1])
f555w_hdu.close()

# ((np.float64(-0.8814290765714752),),
#  np.float64(0.24055442323969883),
#  np.float64(-0.3472747023393392),
#  np.float64(0.12601848622183956))

## F438W

In [ ]:
f438w_data = fits.getdata(f438w)
f438w_data_sliced = f438w_data[3400:3700, 4400:4700]
pixtoarcs = 0.03962
fwhm = 0.11 * 2 # originally found 0.18 arcsec through ds9
f438w_objects = DaoFindObjects(f438w_data_sliced, fwhm=fwhm, pixtoarcs=pixtoarcs,
                               threshold=2, plot=True, vmax=0.09)
print(len(f438w_objects)) # found with 195 sources fwhm=0.10

#investigating the daofind sources and extracting their photometry
f438w_hdu = fits.open(f438w)
apcorr438w, aperr438w, apcorr_ext438w, aperr_ext438w = RunPhots(f438w_hdu, gal="M66", instrument="WFC3",
                                               filter='F438W', fwhm_arcs=fwhm, threshold=2,
                                               num_stars=20, extended_rad=8, reg_correction=[1,1])
f438w_hdu.close()

# ((np.float64(-0.9175731144364969),),
#  np.float64(0.2291917542835823),
#  np.float64(-0.4745254777532202),
#  np.float64(0.15543813531799572))

## F336W

In [ ]:
f336w_data = fits.getdata(f336w)
f336w_data_sliced = f336w_data[3400:3700, 4400:4700]
pixtoarcs = 0.03962
fwhm = 0.12 * 2 # originally found 0.18 arcsec through ds9
f336w_objects = DaoFindObjects(f336w_data_sliced, fwhm=fwhm, pixtoarcs=pixtoarcs,
                               threshold=2, plot=True, vmax=0.05)
print(len(f336w_objects)) # found with 195 sources fwhm=0.10

# investigating the daofind sources and extracting their photometry
f336w_hdu = fits.open(f336w)
apcorr336w, aperr336w, apcorr_ext336w, aperr_ext336w = RunPhots(f336w_hdu, gal="M66", instrument="WFC3",
                                               filter='F336W', fwhm_arcs=fwhm, threshold=2,
                                               num_stars=20, extended_rad=8, reg_correction=[1,1])
f336w_hdu.close()

# ((np.float64(-0.9540754478692428),),
#  np.float64(0.23448030394882666),
#  np.float64(-0.43512525755218445),
#  np.float64(0.13649683194194648))

## F275W

In [ ]:
f275w_data = fits.getdata(f275w)
f275w_data_sliced = f275w_data[4400:5000, 4300:4900]
pixtoarcs = 0.03962
fwhm = 0.12 * 2 # originally found 0.18 arcsec through ds9
f275w_objects = DaoFindObjects(f275w_data_sliced, fwhm=fwhm, pixtoarcs=pixtoarcs,
                               threshold=2, plot=True, vmax=0.015)
print(len(f275w_objects)) # found with 195 sources fwhm=0.10

#investigating the daofind sources and extracting their photometry
f275w_hdu = fits.open(f275w)
apcorr275w, aperr275w, apcorr_ext275w, aperr_ext275w = RunPhots(f275w_hdu, gal="M66", instrument="WFC3",
                                               filter='F275W', fwhm_arcs=fwhm, threshold=2,
                                               num_stars=20, extended_rad=8, reg_correction=[1,1])
f275w_hdu.close()

# ((np.float64(-0.9707309801124901),),
#  np.float64(0.25887846989477437),
#  np.float64(-0.4419304737417187),
#  np.float64(0.1511766174865048))

# Astrometric Corrections

M66_CSC = LoadSources(chandra_hst_dir+"cscresults_M66_renamed.frame")
M66_unique = FindUnique(M66_CSC, header="CSC ID")
WriteReg(M66_unique, outfile=chandra_hst_dir+"M66_cscsources.reg", idheader="CSC ID", 
         radius=5, radunit="arcsec", width=2, color="hotpink")

In [ ]:
M66_calibrators = ["2CXO J112014.8+125814",
                   "2CXO J112018.9+130135",
                   "2CXO J112019.1+125947", # Not sure about this (upper left corner)
                   "2CXO J112019.5+125958", # Not sure about this (upper left corner)
                   "2CXO J112018.8+125924",
                  ] 
print(f"{len(M66_calibrators)} calibrators to match")

M66_calibrators = RemoveElse(M66_unique, keep=M66_calibrators, header='CSC ID')
print(f"{len(M66_calibrators)} calibrators found")

WriteReg(M66_calibrators, outfile=chandra_hst_dir+"M66_calibrators.reg")

In [ ]:
base_coords = GetCoords(hstdir+"M66_daofind_f555w_wfc3_fk5.reg")
cat_coords = [M66_calibrators["RA"].values.tolist(),
              M66_calibrators["Dec"].values.tolist()]

CorrectAstrometry(base_coords, cat_coords, returnshifts=True,
                 savebasereg=chandra_hst_dir+"M66_astrocorrect.reg")

# Input min and max separation: 0, 9.6e-5
# Min and max x residuals: -3e-5, 2.2e-5
# Min and max y residuals: -4.2e-5, 1.6e-5

# RESULTS OF ASTROMETRY CORRECTION

# Median offsets in x and y:
#  -2.018614964072185e-05 3.766607498967289e-05 degrees
#  -0.07267013870659866 0.13559786996282241 arcsecs

# Standard deviation in x and y offsets:
#  1.7643739978865733e-05 1.9961232387211725e-05 degrees
#  0.06351746392391663 0.07186043659396221 arcsecs

#  (np.float64(-2.018614964072185e-05),
#  np.float64(3.766607498967289e-05),
#  np.float64(1.7643739978865733e-05),
#  np.float64(1.9961232387211725e-05))

Assuming that the above astrometric corrections and its outputs are correct, since the standard deviations in the x and y offsets are bigger than their corresponding median offsets, there is not any correction needed between them. 

## Calculating Positional Uncertainties

In [ ]:
sig1, sig2 = CalcPU(M66_CSC, std=[0.06352,0.07186], sig2search='Err Ellipse Major')

In [ ]:
M66_CSC['1Sig'] = sig1  # Saves sig1 to a new header called '1Sig'
M66_CSC['2Sig'] = sig2  # Saves sig2 to a new header called '2Sig'

# The new DataFrame has 1 and 2sig added at the end of the columns list
display(M66_CSC)

M66_CSC.to_csv(chandra_hst_dir+'M66_csc_astrocorrected.frame')

In [ ]:
# Pulling the ID of each unique CSC source
ids = FindUnique(M66_CSC, header="CSC ID")["CSC ID"].values.tolist()

# Building an empty DataFrame, which I will fill below
M66_best = BuildFrame()

for i in ids: # for each unique ID pulled from CSC...
    
    # Search for all instances (i.e. observations) of each source
    Temp = Find(M66_CSC, "CSC ID = " + i)
    
    # Try to avoid sources where counts = NaN (invalid observations) 
    if len(Find(Temp, ["Counts != NaN", "Theta != NaN"])) > 0: 
        Temp = Find(Temp, ["Counts != NaN", "Theta != NaN"])
    
    # Specifically focus on those with a valid number of counts
    if len(Find(Temp, "Counts > 0")) > 0: 
        Temp = Find(Temp, ["Counts > 0"])
        
    # Otherwise, all instances with counts = 0 have the same measurements, 
    # so it doesn't matter which row is chosen for the best radii
    else: pass;
    
    # Take the source with the smallest 2sig. 
    # If there's more than one, take the first on the list.
    Tempbest = Find(Temp, "2Sig =< " + str(min(Temp["2Sig"]))).iloc[:1]
    
    # Add the chosen source observation to the new DataFrame
    M66_best = pd.concat([M66_best, Tempbest], ignore_index=True)

# Saving results to a DataFrame file
# This file contains only rows from M101_CSC that has the 
# best 2sigma radius, based on the search performed above
M66_best.to_csv(chandra_hst_dir+"M66_csc_bestrads.frame")

display(M66_best)

In [ ]:
# Saving the positional uncertainties as region files
WriteReg(M66_best, radius=M66_best['2Sig'].values.tolist(), radunit='arcsec', \
         idheader="CSC ID", outfile=chandra_hst_dir+'M66_bestrads_2sig.reg')
WriteReg(M66_best, radius=M66_best['1Sig'].values.tolist(), radunit='arcsec', \
         outfile=chandra_hst_dir+'M66_bestrads_1sig.reg')g

# Identifying Optical Counterparts to X-ray sources

In [43]:
x,y = GetCoords(hstdir+"M66_daofind_f555w_wfc3_img.reg")
ids = GetIDs(hstdir+"M66_daofind_f555w_wfc3_img.reg")
ra,dec = GetCoords(hstdir+"M66_daofind_f555w_wfc3_fk5.reg")

# Compiling into a single DataFrame
DaoFrame = BuildFrame(headers=['DaoID','X','Y','RA','Dec'], 
                      values=[ids,x,y,ra,dec])
DaoFrame.to_csv(hstdir+"M66_daofind_acs_coords.frame")

Retrieving coordinates from /Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/HST/M66_daofind_f555w_wfc3_img.reg
Retrieving IDs from /Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/HST/M66_daofind_f555w_wfc3_img.reg
Retrieving coordinates from /Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/HST/M66_daofind_f555w_wfc3_fk5.reg


In [45]:
DaoFrame = pd.read_csv(hstdir+"M66_daofind_acs_coords.frame").drop("Unnamed: 0", axis=1)
display(DaoFrame)

,DaoID,X,Y,RA,Dec
0,1,4044.301157,506.383541,170.058899,12.933998
1,2,4071.287505,514.796584,170.058594,12.934091
2,3,4174.768903,607.892406,170.057425,12.935115
3,4,3852.428200,659.309812,170.061065,12.935681
4,5,3885.366892,660.899267,170.060693,12.935699
...,...,...,...,...,...
35153,35154,2559.137495,8946.778618,170.075675,13.026889
35154,35155,2270.289930,8989.284099,170.078938,13.027356
35155,35156,2218.000801,9011.531171,170.079529,13.027601
35156,35157,2269.749207,9011.687347,170.078944,13.027603


In [46]:
# Loading in the original DataFrame containing the best 2sig radii for each source
M66_best = LoadSources(chandra_hst_dir+"M66_csc_bestrads.frame")

# Reading in the newly-saved img region file
xsources, ysources = GetCoords(chandra_hst_dir+"M66_bestrads_2sig_img.reg")
# Adding the image coordinates to the DataFrame
M66_best["X"] = xsources
M66_best["Y"] = ysources

# Convert the radius from arcseconds to pixels. 
# The conversion is 0.05 for ACS/WFC and 0.03962 for WFC3/UVIS
M66_best["2Sig (pix)"] = M66_best["2Sig"] / 0.03962 

# Saving the changes to the DataFrame
M66_best.to_csv(chandra_hst_dir+"M66_csc_bestrads.frame")
display(M66_best)


Reading in sources from /Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/Chandra-HST/M66_csc_bestrads.frame...
Retrieving coordinates from /Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/Chandra-HST/M66_bestrads_2sig_img.reg


,Separation,CSC ID,RA,Dec,Err Ellipse Major,Err Ellipse Minor,Err Ellipse Angle,Significance,Likelihood,conf_flag,...,Theta,HS Ratio,Counts,Counts lolim,Counts hilim,1Sig,2Sig,X,Y,2Sig (pix)
0,0.603512,2CXO J112014.9+125929,170.062428,12.991495,0.311932,0.311917,0.000000,9.567092,TRUE,True,...,0.420311,0.029981,173.048934,154.988228,191.109641,0.115174,0.238746,3732.25780,5731.0323,6.025886
1,1.154310,2CXO J112015.0+125928,170.062878,12.991329,0.317501,0.317501,0.000000,7.701153,TRUE,True,...,0.412041,-0.277327,126.386565,109.441688,142.201784,0.118283,0.248443,3692.38100,5715.9783,6.270647
2,4.249587,2CXO J112014.9+125933,170.062259,12.992633,0.299684,0.298357,5.587902,12.795398,TRUE,False,...,0.488407,0.224859,226.842541,209.119065,244.566017,0.113925,0.235415,3747.18600,5834.4389,5.941826
3,4.484088,2CXO J112015.2+125932X,170.063665,12.992189,3.446421,3.446421,0.000000,39.744111,TRUE,NaN,...,0.466503,NaN,3329.616280,3231.522180,3427.710381,1.725878,3.451755,3622.72450,5794.0764,87.121537
4,5.940189,2CXO J112015.3+125932,170.064119,12.992230,0.413187,0.343140,64.093176,5.142857,TRUE,False,...,0.477466,0.199875,39.229231,31.383385,46.639197,0.154105,0.345133,3582.56790,5797.8225,8.711076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,231.679093,2CXO J112030.5+130013,170.127457,13.003663,1.402157,0.820192,1.489236,3.575758,TRUE,False,...,3.984317,0.201124,18.342335,13.057255,23.316527,0.439176,1.071654,-2024.96930,6837.4179,27.048301
84,237.496980,2CXO J112008.5+130307,170.035399,13.051914,0.774353,0.647498,139.851620,4.000000,TRUE,False,...,4.338261,0.463460,24.421586,18.316189,30.526982,0.423189,1.022475,6124.79330,11221.0620,25.807047
85,240.218881,2CXO J112019.6+130320,170.081736,13.055570,0.563631,0.427724,157.623228,10.427182,TRUE,False,...,4.414767,0.763273,150.478203,136.047662,164.908744,0.194439,0.404728,2023.27800,11553.1970,10.215239
86,246.797558,2CXO J112027.8+125649,170.116113,12.946999,1.754478,1.143646,33.705932,3.294118,TRUE,False,...,3.872333,-0.408495,17.434501,12.141885,22.727118,0.436688,1.067144,-1021.79190,1688.5554,26.934484


In [59]:
DaoCleanFrame = DaoClean(daosources=DaoFrame, sources=M66_best, sourceid="CSC ID", 
                         coordsys="img", coordheads=['X','Y'], radheader="2Sig (pix)",
                         wiggle=0.5) 


# Renaming the DaoID header to "F555W ID", to match the output of the other filters
DaoCleanFrame = DaoCleanFrame.rename(columns={'DaoID': 'F555W ID'})

display(DaoCleanFrame)

WriteReg(DaoCleanFrame, coordsys='fk5', width=2, radius=0.2, radunit='arcsec',
         outfile=chandra_hst_dir+'M66_daoclean_f555w_wfc3_fk5.reg')

Cleaning DAOFind sources. This will take a few minutes. Please wait.. 
DONE WITH CLEANING. CREATING DATAFRAME...


,F555W ID,X,Y,RA,Dec,CSC ID
0,18393,3728.392237,5731.691614,170.062466,12.991505,2CXO J112014.9+125929
1,18431,3730.3829,5734.693669,170.062444,12.991539,2CXO J112014.9+125929
2,18254,3690.922608,5713.903685,170.06289,12.99131,2CXO J112015.0+125928
3,18303,3693.339708,5721.089782,170.062862,12.991389,2CXO J112015.0+125928
4,19206,3746.7145,5828.6536,170.06226,12.992573,2CXO J112014.9+125933
...,...,...,...,...,...,...
236,3994,4869.455824,3777.647306,170.049579,12.97,2CXO J112011.8+125811
237,34886,3768.771693,8363.063045,170.062011,13.020465,2CXO J112014.9+130113
238,1804,4463.71084,2888.783967,170.054162,12.960218,2CXO J112013.0+125736
239,1476,3375.849092,2661.999469,170.066447,12.957722,2CXO J112015.9+125727


Saving /Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/Chandra-HST/M66_daoclean_f555w_wfc3_fk5.reg
/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/Chandra-HST/M66_daoclean_f555w_wfc3_fk5.reg saved!


# Cross-referencing sources across filters

In [53]:
cd(hstdir)
regions = ["M66_daofind_f275w_wfc3_img.reg",
           "M66_daofind_f336w_wfc3_img.reg",
           "M66_daofind_f438w_wfc3_img.reg",
           "M66_daofind_f814w_wfc3_img.reg"]

DaoCleanMatch = Crossref(DaoCleanFrame, regions=regions, catalogs=['F275W', 'F336W', 'F438W', 'F814W'],
                        coordheads=['X','Y'], sourceid="F555W ID",
                        outfile="M66_daoclean_matches.frame")

DaoCleanMatch

Finding cross-references between sources. This will take a few minutes. Please wait.. 
DONE WITH CLEANING. CREATING DATAFRAME...


,X,Y,F555W ID,F275W ID,F336W ID,F438W ID,F814W ID
0,3728.392237,5731.691614,18393,NaN,95500,34799,17413
1,3730.382900,5734.693669,18431,NaN,NaN,NaN,17413
2,3690.922608,5713.903685,18254,NaN,95118,34583,NaN
3,3693.339708,5721.089782,18303,NaN,NaN,NaN,NaN
4,3746.714500,5828.653600,19206,NaN,NaN,35989,NaN
...,...,...,...,...,...,...,...
257,3768.771693,8363.063045,34886,203812,157738,59483,NaN
258,4463.710840,2888.783967,1804,48203,NaN,NaN,1718
259,3375.849092,2661.999469,1476,NaN,25603,5443,1413
260,3375.849092,2661.999469,1476,NaN,25753,NaN,NaN


In [54]:
# Adding "CSC ID" to columns in DaoCleanMatch
DaoCleanMatch["CSC ID"] = None

# Adding the CSC IDs of each source the daosources are associated with
for i in range(len(DaoCleanMatch)): 
    # Searches for the CSC ID associated with each F555W ID (DaoID in DaoCleanFrame)
    tempid = DaoCleanMatch["F555W ID"][i] 
    tempcsc = Find(DaoCleanFrame, "F555W ID = " + str(tempid))["CSC ID"][0]
    DaoCleanMatch["CSC ID"][i] = tempcsc
    
# Updating the DataFrame
DaoCleanMatch.to_csv(hstdir+"M66_daoclean_matches.frame")
display(DaoCleanMatch)

,X,Y,F555W ID,F275W ID,F336W ID,F438W ID,F814W ID,CSC ID
0,3728.392237,5731.691614,18393,NaN,95500,34799,17413,2CXO J112014.9+125929
1,3730.382900,5734.693669,18431,NaN,NaN,NaN,17413,2CXO J112014.9+125929
2,3690.922608,5713.903685,18254,NaN,95118,34583,NaN,2CXO J112015.0+125928
3,3693.339708,5721.089782,18303,NaN,NaN,NaN,NaN,2CXO J112015.0+125928
4,3746.714500,5828.653600,19206,NaN,NaN,35989,NaN,2CXO J112014.9+125933
...,...,...,...,...,...,...,...,...
257,3768.771693,8363.063045,34886,203812,157738,59483,NaN,2CXO J112014.9+130113
258,4463.710840,2888.783967,1804,48203,NaN,NaN,1718,2CXO J112013.0+125736
259,3375.849092,2661.999469,1476,NaN,25603,5443,1413,2CXO J112015.9+125727
260,3375.849092,2661.999469,1476,NaN,25753,NaN,NaN,2CXO J112015.9+125727


In [57]:
# For each X-ray source, number the associated stars starting from 1.
tempcsc = DaoCleanMatch["CSC ID"][0] # Keeping track of the current X-ray source
DaoCleanMatch["StarID"] = None # Storing the star number for each X-ray source

starno = 1 # Star number counter

for i in range(len(DaoCleanMatch)): 
    # If this is a new CSC ID, save it to tempcsc and restart starno counter
    if tempcsc != DaoCleanMatch["CSC ID"][i]: 
        tempcsc = DaoCleanMatch["CSC ID"][i]
        starno = 1
    DaoCleanMatch["StarID"][i] = starno
    starno += 1 # adds one to the starno counter

# Saving the candidate region file with new numberings
WriteReg(DaoCleanMatch, coordsys="image", coordheads=["X","Y"], 
         idheader="StarID", width=2, fontsize=12,
         outfile=chandra_hst_dir+"M66_XRB_candidates.reg")

# DaoCleanMatch has now renumbered each star associated with 
# an X-ray source for simplicity
display(DaoCleanMatch)

Saving /Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/Chandra-HST/M66_XRB_candidates.reg
/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/Chandra-HST/M66_XRB_candidates.reg saved!


,X,Y,F555W ID,F275W ID,F336W ID,F438W ID,F814W ID,CSC ID,StarID
0,3728.392237,5731.691614,18393,NaN,95500,34799,17413,2CXO J112014.9+125929,1
1,3730.382900,5734.693669,18431,NaN,NaN,NaN,17413,2CXO J112014.9+125929,2
2,3690.922608,5713.903685,18254,NaN,95118,34583,NaN,2CXO J112015.0+125928,1
3,3693.339708,5721.089782,18303,NaN,NaN,NaN,NaN,2CXO J112015.0+125928,2
4,3746.714500,5828.653600,19206,NaN,NaN,35989,NaN,2CXO J112014.9+125933,1
...,...,...,...,...,...,...,...,...,...
257,3768.771693,8363.063045,34886,203812,157738,59483,NaN,2CXO J112014.9+130113,1
258,4463.710840,2888.783967,1804,48203,NaN,NaN,1718,2CXO J112013.0+125736,1
259,3375.849092,2661.999469,1476,NaN,25603,5443,1413,2CXO J112015.9+125727,1
260,3375.849092,2661.999469,1476,NaN,25753,NaN,NaN,2CXO J112015.9+125727,2


## Incorporating Other Catalogs

### Cluster Catalog
The cluster catalog being used below can be found from the PHANGS-CAT website, [here](https://archive.stsci.edu/hlsp/phangs/phangs-cat). Note that the catalog above was observed through wfc3. There's another one available taken through acs, [here](https://archive.stsci.edu/hlsp/phangs.html#hst_image_products_table). However, since I am using the wfc3 instrument for all my analysis here, I am using the wfc3 catalog only.

In [61]:
# Read a BinTableHDU
cluster_hdu = fits.open('/Users/undergradstudent/Downloads/ngc3627/cluster/hlsp_phangs-cat_hst_wfc3_ngc3627_multi_v1_cluster-human-class12.fits')
clustertbl = QTable(cluster_hdu[1].data)
clustertbl

INDEX,ID_PHANGS_CLUSTERS,PHANGS_X,PHANGS_Y,PHANGS_RA,PHANGS_DEC,PHANGS_CLUSTER_CLASS_HUMAN,PHANGS_CLUSTER_CLASS_ML_VGG,PHANGS_CLUSTER_CLASS_ML_VGG_QUAL,PHANGS_F275W_VEGA_TOT,PHANGS_F275W_VEGA_TOT_ERR,PHANGS_F336W_VEGA_TOT,PHANGS_F336W_VEGA_TOT_ERR,PHANGS_F438W_VEGA_TOT,PHANGS_F438W_VEGA_TOT_ERR,PHANGS_F555W_VEGA_TOT,PHANGS_F555W_VEGA_TOT_ERR,PHANGS_F814W_VEGA_TOT,PHANGS_F814W_VEGA_TOT_ERR,PHANGS_F275W_mJy_TOT,PHANGS_F275W_mJy_TOT_ERR,PHANGS_F336W_mJy_TOT,PHANGS_F336W_mJy_TOT_ERR,PHANGS_F438W_mJy_TOT,PHANGS_F438W_mJy_TOT_ERR,PHANGS_F555W_mJy_TOT,PHANGS_F555W_mJy_TOT_ERR,PHANGS_F814W_mJy_TOT,PHANGS_F814W_mJy_TOT_ERR,PHANGS_CI,PHANGS_AGE_MINCHISQ,PHANGS_AGE_MINCHISQ_ERR,PHANGS_MASS_MINCHISQ,PHANGS_MASS_MINCHISQ_ERR,PHANGS_EBV_MINCHISQ,PHANGS_EBV_MINCHISQ_ERR,PHANGS_REDUCED_MINCHISQ,NO_DETECTION_FLAG
int64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,14,6614.89,3239.51,170.06613550163783,12.939565672437887,1.0,2,0.9,20.64478245728906,0.02659657396351643,20.93979017654412,0.030188736564226437,22.250655300429276,0.031346415447021464,22.27859441730564,0.016875740045112282,22.11064764254441,0.03901893634778962,0.004894710575974825,0.00012002236400823634,0.005149012301279204,0.00014331060515183322,0.0051853476618246,0.00014985637672020489,0.004488303673158532,6.983200680382467e-05,0.003450014679669524,0.00012410973040724197,1.4406957552854887,1.0,1.0,8719.629993702069,2694.2879321260407,0.31,0.03999999999999998,0.15571900335067532,0.0
2,46,7284.4,3396.34,170.05857517129016,12.94129168906165,1.0,1,0.8,24.105332415990546,0.37037020453020186,22.714208348075548,0.08964678639377795,22.519651769070073,0.027672450452808608,21.668955157650842,0.009359068435449567,20.217371142107172,0.008528855229104975,0.0002020724158307192,6.900054767200979e-05,0.0010045155423948492,8.302346398363295e-05,0.004047429496485954,0.00010326101122819036,0.007869333245767959,6.790160231908834e-05,0.01973020252367809,0.00015514284407774865,1.6525453388887816,2440.0,1412.0,435789.158149563,132243.1815034949,0.15,0.17,0.6757897715262302,0.0
3,55,6620.99,3457.63,170.0660666598804,12.941966204158541,1.0,1,1.0,22.281918510623996,0.07737770478364059,22.22169363119243,0.0616848745282317,22.46988424208646,0.027246659028787583,22.442268805930635,0.014600834523023043,21.98812430101322,0.02959330849684471,0.001083608980805747,7.730327650696346e-05,0.0015811116822482226,8.991884008849886e-05,0.0042372716090699316,0.00010644101884595906,0.003860223743174932,5.196364647241765e-05,0.003862160160097809,0.00010537395092536263,1.5549336132831346,182.0,28.0,17467.385090088555,2360.828541606472,0.0,0.01,0.7982185920792995,0.0
4,68,6698.34,3482.21,170.06519319571672,12.942236733359024,1.0,1,1.0,20.82626233865763,0.02611063539413167,20.81526062516283,0.022289843866434313,21.009603537555332,0.010733207751301713,20.91452051154636,0.006500915802188716,20.38050871921674,0.009989056907406837,0.004141284477435931,9.969238081274846e-05,0.005774784072017913,0.00011867308687031326,0.01626292222205026,0.00016093021202070686,0.015765646894598556,9.449216496329995e-05,0.01697760840966834,0.00015635452786221064,1.7234222094582332,210.0,0.0,80145.11041876292,9281.609214165932,0.01,0.03,0.181573970260038,0.0
5,90,6745.89,3520.91,170.06465624597297,12.942662654573844,1.0,1,1.0,22.925218259744764,0.13452151333237802,22.27913233033862,0.06630949310249956,22.105464738627294,0.022063156905752657,21.881795724754113,0.012211765860952238,21.189383833152473,0.016662480049139786,0.0005991736362875742,7.431113649301165e-05,0.0014996401795385338,9.167950055632037e-05,0.005927261726618868,0.00012056776794817406,0.006468472112226721,7.282653409476053e-05,0.008059840858327328,0.00012381565857698675,1.8696595576630655,500.0,151.0,55569.664341832664,1130.107884178833,0.0,0.1,

In [ ]:
regions = []
radius = 0.00005 * u.deg       # change this depending on the desired size of the radius
for i in range(len(clustertbl)):
    # create sky coordinates
    center_sky = SkyCoord(ra=clustertbl['PHANGS_RA'][i], dec=clustertbl['PHANGS_DEC'][i],
                          unit='deg', frame='icrs').transform_to(FK5)
    sky_reg = CircleSkyRegion(center=center_sky, radius=radius)
    regions.append(sky_reg)

regs = Regions(regions)
regs.write(chandra_hst_dir+cluster_catalog_.reg", format='ds9')
regs

<Regions([<CircleSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.06614071, 12.93956223)>, radius=5e-05 deg)>, <CircleSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.05858038, 12.94128824)>, radius=5e-05 deg)>, <CircleSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.06607187, 12.94196276)>, radius=5e-05 deg)>, <CircleSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.06519841, 12.94223329)>, radius=5e-05 deg)>, <CircleSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.06466146, 12.94265921)>, radius=5e-05 deg)>, <CircleSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.0652489, 12.94323424)>, radius=5e-05 deg)>, <CircleSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.06312355, 12.94414244)>, radius=5e-05 deg)>, <CircleSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in d

In [ ]:
# Add a few degrees to the center coordinate of the source
# so that the region that is created would be above the source
clustercopy = clustertbl.copy()
clustercopy['PHANGS_DEC'] = clustercopy['PHANGS_DEC'] + 0.00005

regions = []
radius = 0.00008 * u.deg
for i in range(len(clustertbl)):
    center_sky = SkyCoord(ra=clustertbl['PHANGS_RA'][i], dec=clustercopy['PHANGS_DEC'][i],
                          unit='deg', frame='icrs').transform_to(FK5)
    sky_reg = TextSkyRegion(center=center_sky, text=clustertbl['ID_PHANGS_CLUSTERS'][i])
    regions.append(sky_reg)

regs = Regions(regions)
regs.write(chandra_hst_dir+"cluster_catalog_text.reg", format='ds9')
regs


<Regions([<TextSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.06614071, 12.93961223)>, text=np.int64(14))>, <TextSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.05858038, 12.94133824)>, text=np.int64(46))>, <TextSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.06607187, 12.94201276)>, text=np.int64(55))>, <TextSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.06519841, 12.94228329)>, text=np.int64(68))>, <TextSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.06466146, 12.94270921)>, text=np.int64(90))>, <TextSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.0652489, 12.94328424)>, text=np.int64(112))>, <TextSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (170.06312355, 12.94419244)>, text=np.int64(148))>, <TextSkyRegion(center=<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    

### Supernova Remnants (SNRs)